转换成照片

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

# 创建目标文件夹
os.makedirs('readyrawdata', exist_ok=True)

for filename in range(1223, 5223):
    file_path = f'rawdata/{filename}'
    try:
        with open(file_path, 'rb') as fid:
            I = np.fromfile(fid, dtype=np.uint8)

        # 判断是否为完整信息的图像
        if len(I) == 128*128:
            # 逆时针旋转90度
            rotated_img = np.rot90(I.reshape(128, 128), k=1)

            # 保存图像到目标文件夹
            new_file_path = f'readyrawdata/{filename}.png'
            plt.imsave(new_file_path, rotated_img, cmap='gray', vmin=0, vmax=255)
    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")


Error processing 1228: [Errno 2] No such file or directory: 'rawdata/1228'
Error processing 1232: [Errno 2] No such file or directory: 'rawdata/1232'
Error processing 1808: [Errno 2] No such file or directory: 'rawdata/1808'
Error processing 4056: [Errno 2] No such file or directory: 'rawdata/4056'
Error processing 4135: [Errno 2] No such file or directory: 'rawdata/4135'
Error processing 4136: [Errno 2] No such file or directory: 'rawdata/4136'
Error processing 5004: [Errno 2] No such file or directory: 'rawdata/5004'


生成带确实信息的npy文件

In [2]:
import numpy as np

# 读取faceDR文件
data = []
with open('faceDR', 'r') as file:
    for line in file:
        if line.startswith(' '):
            line = line.strip()
            # 提取第一列和第三列数据
            parts = line.split()
            column1 = parts[0]
            column3 = parts[2]
            # 去掉括号和引号
            column3 = column3.replace(")", "").replace("'", "")
            # 将第一列和处理后的第三列数据添加到data列表中
            data.append((column1, column3))

# 转换数据为NumPy数组
arr = np.array(data, dtype=[('第一列', 'U10'), ('第三列', 'U10')])

# 保存为.npy文件（保存在当前路径下）
np.save('face_train.npy', arr)
data2 = np.load('face_train.npy')
print(data2)

[('1223', 'male') ('1224', 'male') ('1225', 'male') ... ('3220', 'female')
 ('3221', 'female') ('3222', 'female')]


/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/numpy/lib/format.py:425: UserWarning: Stored array in format 3.0. It can only be read by NumPy >= 1.17
  header = _wrap_header_guess_version(header)


In [3]:
import numpy as np

# 读取faceDS文件
data = []
with open('faceDS', 'r') as file:
    for line in file:
        if line.startswith(' '):
            line = line.strip()
            # 提取第一列和第三列数据
            parts = line.split()
            column1 = parts[0]
            column3 = parts[2]
            # 去掉括号和引号
            column3 = column3.replace(")", "").replace("'", "")
            # 将第一列和处理后的第三列数据添加到data列表中
            data.append((column1, column3))

# 转换数据为NumPy数组
arr = np.array(data, dtype=[('第一列', 'U10'), ('第三列', 'U10')])

# 保存为.npy文件（保存在当前路径下）
np.save('face_test.npy', arr)
data2 = np.load('face_test.npy')
print(data2)

[('3223', 'female') ('3224', 'female') ('3225', 'male') ...
 ('5220', 'female') ('5221', 'female') ('5222', 'female')]


In [4]:
import numpy as np

# 加载.npy文件
data1 = np.load('face_train.npy')
data2 = np.load('face_test.npy')
# 打印内容
print(data1)
print(data2)

[('1223', 'male') ('1224', 'male') ('1225', 'male') ... ('3220', 'female')
 ('3221', 'female') ('3222', 'female')]
[('3223', 'female') ('3224', 'female') ('3225', 'male') ...
 ('5220', 'female') ('5221', 'female') ('5222', 'female')]


去除含有缺失信息的数据

In [5]:
import numpy as np

# 加载.npy文件
data = np.load('face_train.npy')

# 创建一个新列表，存储不包含缺失值的条目
data_cleaned = [entry for entry in data if not any('descriptor' in field for field in entry)]

# 将处理后的数据保存为.npy文件
np.save('Train_data.npy', np.array(data_cleaned))

# 加载.npy文件
data = np.load('face_test.npy')

# 创建一个新列表，存储不包含缺失值的条目
data_cleaned = [entry for entry in data if not any('descriptor' in field for field in entry)]

# 将处理后的数据保存为.npy文件
np.save('Test_data.npy', np.array(data_cleaned))

调整数据集

In [6]:
import numpy as np

# 读取unprocessed_expression_train.npy和unprocessed_expression_test.npy文件
train_data = np.load('Train_data.npy')
test_data = np.load('Test_data.npy')

# 计算要提取的数据的数量
num_extract = int(len(train_data) * 1 )

# 提取unprocessed_expression_train.npy的末尾三分之二数据
extracted_data = train_data[-num_extract:]

# 将extracted_data插入到unprocessed_expression_test.npy的开头
combined_data = np.concatenate((extracted_data, test_data), axis=0)

# 更新unprocessed_expression_train.npy，删除已提取的数据
updated_train_data = train_data[:-num_extract]

# 保存合并后的数据到unprocessed_expression_test.npy文件
np.save('Test_data.npy', combined_data)

# 保存更新后的unprocessed_expression_train.npy文件
np.save('Train_data.npy', updated_train_data)
import numpy as np

# 加载.npy文件
data1 = np.load('Train_data.npy')
data2 = np.load('Test_data.npy')
# 打印内容
print(data1)
print(data2)


[]
[('1223', 'male') ('1224', 'male') ('1225', 'male') ... ('5220', 'female')
 ('5221', 'female') ('5222', 'female')]


In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18
from torchvision.transforms import ToTensor, Normalize
from PIL import Image
import os

In [8]:
# 自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, ids, genders, transform=None):
        self.ids = ids
        self.genders = genders
        self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        image_id = self.ids[idx]
        gender = self.genders[idx]
        image_path = os.path.join("readyrawdata", f"{image_id}.png")

        if not os.path.exists(image_path):
            return None, None

        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = 0 if gender == '男' else 1

        return image, label


In [9]:
# 加载Train_data.npy和Test_data.npy
'''
train_data = np.load('Train_data.npy')
test_data = np.load('Test_data.npy')import numpy as np
'''
# 加载Test_data.npy数据
data = np.load('Test_data.npy')

# 随机打乱数据
np.random.shuffle(data)

# 计算抽取一半数据的索引位置
half = len(data) // 2

# 分割数据为训练集和测试集
train_data = data[:half]
test_data = data[half:]

In [10]:
# 提取编号和性别信息
train_ids = train_data['第一列'].astype(str)
train_genders = train_data['第三列'].astype(str)
test_ids = test_data['第一列'].astype(str)
test_genders = test_data['第三列'].astype(str)
print(test_genders)

['male' 'female' 'male' ... 'female' 'male' 'male']


In [11]:
# 创建训练集数据集对象
train_dataset = CustomDataset(train_ids, train_genders, transform=ToTensor())
print(len(train_dataset))


1996


In [12]:
# 创建测试集数据集对象
test_dataset = CustomDataset(test_ids, test_genders, transform=ToTensor())
print(len(test_dataset))


1997


In [13]:
# 删除缺失图像的索引
train_dataset.ids = [image_id for image_id in train_dataset.ids if image_id is not None]
train_dataset.genders = [gender for gender, image_id in zip(train_dataset.genders, train_dataset.ids) if image_id is not None]

test_dataset.ids = [image_id for image_id in test_dataset.ids if image_id is not None]
test_dataset.genders = [gender for gender, image_id in zip(test_dataset.genders, test_dataset.ids) if image_id is not None]
#print(test_dataset.ids)
print(len(test_dataset.ids ))

1997


In [14]:
def collate_fn(batch):
    """
    自定义数据集合并函数

    参数:
        batch (list): 批次样本列表

    返回:
        tensor: 合并后的批次数据
    """
    batch = [data for data in batch if data[0] is not None]
    return torch.utils.data.dataloader.default_collate(batch)

# 创建训练数据加载器
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, collate_fn=collate_fn)

# 创建测试数据加载器
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4, collate_fn=collate_fn)



In [15]:
print(train_loader)

In [16]:
# 创建模型
model = resnet18(pretrained=True)
model.fc = nn.Linear(512, 2)

# 将模型移至设备（GPU 或 CPU）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/ma-user/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [17]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=2)


In [18]:
# 定义训练函数
       # print(f'Epoch {epoch + 1}/{num_epochs} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_accuracy:.4f}')打印训练信息

import torch
import torch.nn as nn
import torch.optim as optim

# 定义训练函数
def train_model(model, criterion, optimizer, scheduler, num_epochs, early_stopping_patience):
    model.train()

    best_accuracy = 0.0
    best_model = None
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0

        for images, labels in train_loader:
            # 将数据移至设备（GPU 或 CPU）
            images = images.to(device)
            labels = labels.to(device)

            # 梯度归零
            optimizer.zero_grad()

            # 前向传播
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            # 计算损失
            loss = criterion(outputs, labels)

            # 添加正则化项
            l2_regularization = 0.0
            for param in model.parameters():
                l2_regularization += torch.norm(param, 2)

            loss += 0.1 * l2_regularization

            # 反向传播及优化
            loss.backward()
            optimizer.step()

            # 统计损失和准确率
            running_loss += loss.item() * images.size(0)
            correct_predictions += torch.sum(predicted == labels.data)

        if scheduler is not None:
            scheduler.step()

        # 计算平均损失和准确率
        epoch_loss = running_loss / len(train_dataset)
        epoch_accuracy = correct_predictions.double() / len(train_dataset)

        # 打印训练信息
        print(f'Epoch {epoch + 1}/{num_epochs} | Loss: {epoch_loss:.4f}| Accuracy: {epoch_accuracy:.4f} ')

        # 验证模型性能并进行早停法判断
        #test_accuracy = test_model(model, criterion, test_loader)

        if epoch_accuracy > best_accuracy:
            best_accuracy = epoch_accuracy
            best_model = model
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= early_stopping_patience:
            print('Early stopping at epoch {epoch + 1}')
            break

    return best_model

# 调用训练函数进行训练
num_epochs = 3
early_stopping_patience = 1
best_model = train_model(model, criterion, optimizer, scheduler, num_epochs, early_stopping_patience)



Epoch 1/3 | Loss: 46.9952| Accuracy: 0.9739 
Epoch 2/3 | Loss: 46.8487| Accuracy: 0.9995 
Epoch 3/3 | Loss: 46.8148| Accuracy: 0.9995 
Early stopping at epoch {epoch + 1}


In [19]:
# 定义测试函数
def test_model(model, criterion, data_loader):
    model.eval()

    correct_predictions = 0

    with torch.no_grad():
        for images, labels in data_loader:
            # 将数据移至设备（GPU 或 CPU）
            images = images.to(device)
            labels = labels.to(device)

            # 前向传播
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            # 统计准确率
            correct_predictions += torch.sum(predicted == labels.data)

    accuracy = correct_predictions.double() / len(data_loader.dataset)

    print(f'Test Accuracy: {accuracy:.4f}')

    return accuracy


In [20]:
# 调用测试函数
test_model(best_model, criterion, test_loader)

Test Accuracy: 0.9995


tensor(0.9995, dtype=torch.float64)